<a href="https://colab.research.google.com/github/Traivok/tcc/blob/develop/scripts/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! nvidia-smi
! pip install ultralytics --upgrade
! pip install wandb
! pip install roboflow
! pip install scikit-learn
! pip install python-dotenv

Sun Nov 17 16:29:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import ultralytics
import warnings
import torch
import os

In [6]:
%matplotlib inline
ultralytics.checks()

Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.8/235.7 GB disk)


In [7]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

from dotenv import load_dotenv
load_dotenv()

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

if IN_COLAB:
  from google.colab import userdata
  ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')



In [8]:
# Check if running on a MacBook with M1 chip
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) for hardware acceleration on MacBook M1.")
# Check if CUDA is available
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA for hardware acceleration.")
else:
    device = torch.device("cpu")
    warnings.warn("Using CPU for computation. This may be slower than using MPS or CUDA.", UserWarning)

Using CUDA for hardware acceleration.


In [9]:
base_path = os.getcwd()
print(f"Current working directory: {base_path}")

# Example usage with Ultralytics
model_list =  [
    #  'yolov3u.pt',
    #  'yolov5mu.pt',
     'yolov8m.pt',
    #  'yolov9m.pt',
    #  'yolov10m.pt',
    #  'yolo11m.pt',
]

for model_name in model_list:
        print(model_name)
        model = ultralytics.YOLO(model_name).to(device)

Current working directory: /content
yolov8m.pt


In [10]:
from roboflow import Roboflow

print(f"Roboflow API key: {ROBOFLOW_API_KEY}")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("ze").project("tcc-bbcjn")
version = project.version(1)
dataset = version.download("yolov8")

Roboflow API key: SpzA9wCLPkImtTIJ3srw
loading Roboflow workspace...
loading Roboflow project...


In [13]:
! wandb login $WANDB_API_KEY --verify

wandb: Currently logged in as: jraf (jraf-universidade-federal-de-pernambuco). Use `wandb login --relogin` to force relogin


In [12]:
from google.colab import drive
import shutil

# Monta o Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback
from ultralytics import YOLO

DATASET = dataset.location + "/data.yaml"
MODEL_FILENAME=model_list[0] ## should be yolov8
PATIENCE=30
EPOCHS=100
IMGSZ=(640, 640)

MODEL_NAME = os.path.splitext(MODEL_FILENAME)[0]
RUN_NAME=f'tcc_{MODEL_NAME}'

model = YOLO(MODEL_FILENAME)

add_wandb_callback(model, enable_model_checkpointing=True)
wandb.init(project=RUN_NAME)

results = model.train(
    task='detect',
    data=DATASET,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=0.9,
    patience=10,
    project='tcc',
    name=RUN_NAME,
    save=True,
    save_period=5,
    plots=True,
    cache=True,
    seed=0)

wandb.finish()

Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/tcc-1/data.yaml, epochs=100, time=None, patience=10, batch=32, imgsz=(640, 640), save=True, save_period=5, cache=True, device=None, workers=8, project=tcc, name=tcc_yolov8m6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning /content/tcc-1/train/labels.cache... 3684 images, 63 backgrounds, 0 corrupt: 100%|██████████| 3684/3684 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 327, len(boxes) = 7670. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
train: 6.3GB RAM required to cache images with 50% safety margin but only 1.9/12.7GB available, not caching images ⚠️


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/tcc-1/valid/labels.cache... 826 images, 10 backgrounds, 0 corrupt: 100%|██████████| 826/826 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 61, len(boxes) = 1310. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.9GB RAM): 100%|██████████| 826/826 [00:06<00:00, 123.81it/s]


In [15]:
# Define o caminho de destino no Google Drive
destination_dir = f'/content/drive/My Drive/tcc/'
# destination_path = os.path.join(destination_dir, 'best.pt')


# Verifica se o diretório existe; se não, cria-o
import os
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copia o arquivo para o destino no Google Drive
from distutils.dir_util import copy_tree

copy_tree(f'/content/tcc', destination_dir)

print("Arquivo enviado para o Google Drive com sucesso!")

Arquivo enviado para o Google Drive com sucesso!
